# Gene Ontology (GO) Biological Process

Author: Zachary Flamholz  
Date: 06-2018  
Database: http://flybase.org/   
Data: http://geneontology.org/gene-associations/gene_association.fb.gz    
Companion file: http://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Invertebrates/Drosophila_melanogaster.gene_info.gz, http://snapshot.geneontology.org/ontology/go-basic.obo    

In [1]:
import numpy as np
import pandas as pd
import sys, datetime
import goenrich
import scipy.stats as stat

# Versions of Modules in Use

In [2]:
%load_ext version_information
%version_information numpy, pandas

Software versions
Python 3.6.5 64bit [GCC 4.2.1 Compatible Apple LLVM 9.1.0 (clang-902.0.39.1)]
IPython 6.4.0
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.14.5
pandas 0.23.1
Wed Jun 27 11:05:15 2018 EDT

## read in data

In [4]:
df = pd.read_csv('input/gene_association.fb', sep='`', skiprows=16, header=None)

In [5]:
df.head()

,0
0,FB\tFBgn0043467\t064Ya\t\tGO:0048149\tFB:FBrf0...
1,FB\tFBgn0043467\t064Ya\t\tGO:0005575\tFB:FBrf0...
2,FB\tFBgn0043467\t064Ya\t\tGO:0003674\tFB:FBrf0...
3,FB\tFBgn0026615\t10-4\t\tGO:0008150\tFB:FBrf01...
4,FB\tFBgn0026615\t10-4\t\tGO:0003674\tFB:FBrf01...


In [6]:
## code copied from Gene Ontology (GO) Biological Process by Moshe Silverstein 

matrix = np.chararray((df.shape[0], 17), itemsize=150, unicode=True)

for i, row in enumerate(df.itertuples()):
    
    progressPercent = ((i+1)/len(df.index))*100

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df.index)))
    sys.stdout.flush()
    
    lst = row[1].split('\t')

    matrix[i, :] = lst

df_clean = pd.DataFrame(data=matrix)

In [7]:
df_clean.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,FB,FBgn0043467,064Ya,,GO:0048149,FB:FBrf0131396|PMID:11086999,IMP,,P,064Ya,,gene_product,taxon:7227,20060803,FlyBase,,
1,FB,FBgn0043467,064Ya,,GO:0005575,FB:FBrf0159398|GO_REF:0000015,ND,,C,064Ya,,gene_product,taxon:7227,20060803,FlyBase,,
2,FB,FBgn0043467,064Ya,,GO:0003674,FB:FBrf0159398|GO_REF:0000015,ND,,F,064Ya,,gene_product,taxon:7227,20060803,FlyBase,,
3,FB,FBgn0026615,10-4,,GO:0008150,FB:FBrf0159398|GO_REF:0000015,ND,,P,10-4,,gene_product,taxon:7227,20060803,FlyBase,,
4,FB,FBgn0026615,10-4,,GO:0003674,FB:FBrf0159398|GO_REF:0000015,ND,,F,10-4,,gene_product,taxon:7227,20060803,FlyBase,,
5,FB,FBgn0070057,11,,GO:0005575,FB:FBrf0159398|GO_REF:0000015,ND,,C,11,#1,gene_product,taxon:7227,20060803,FlyBase,,
6,FB,FBgn0070057,11,,GO:0003674,FB:FBrf0159398|GO_REF:0000015,ND,,F,11,#1,gene_product,taxon:7227,20060803,FlyBase,,
7,FB,FBgn0010339,128up,,GO:0005525,FB:FBrf0067209|PMID:8121394,IDA,,F,upstream of RpIII128,CG8340|GTP-bp|Gtp-bp|X71866,protein,taxon:7227,20060803,FlyBase,,
8,FB,FBgn0020238,14-3-3epsilon,,GO:0000077,FB:FBrf0127025|PMID:10733527,IMP,,P,14-3-3epsilon,14-3-3|14-3-3 epsilon|14-3-3 protein epsilon|1...,protein,taxon:7227,20060803,FlyBase,,
9,FB,FBgn0020238,14-3-3epsilon,,GO:0000077,FB:FBrf0130073|PMID:10908583,TAS,,P,14-3-3epsilon,14-3-3|14-3-3 epsilon|14-3-3 protein epsilon|1...,protein,taxon:7227,20060803,FlyBase,,


In [8]:
columns = ["DB", "DB gene ID", "Gene symbol", "Qualifier", "GO ID", "Reference", "Evidence code", "Evidence from", "GO class", "attribute", "Locus tag", "gene/protein", "tax id", "date", "Assigned by", "additional information", "empty"]

In [9]:
df_clean.columns = columns

In [10]:
df_clean.head()

,DB,DB gene ID,Gene symbol,Qualifier,GO ID,Reference,Evidence code,Evidence from,GO class,attribute,Locus tag,gene/protein,tax id,date,Assigned by,additional information,empty
0,FB,FBgn0043467,064Ya,,GO:0048149,FB:FBrf0131396|PMID:11086999,IMP,,P,064Ya,,gene_product,taxon:7227,20060803,FlyBase,,
1,FB,FBgn0043467,064Ya,,GO:0005575,FB:FBrf0159398|GO_REF:0000015,ND,,C,064Ya,,gene_product,taxon:7227,20060803,FlyBase,,
2,FB,FBgn0043467,064Ya,,GO:0003674,FB:FBrf0159398|GO_REF:0000015,ND,,F,064Ya,,gene_product,taxon:7227,20060803,FlyBase,,
3,FB,FBgn0026615,10-4,,GO:0008150,FB:FBrf0159398|GO_REF:0000015,ND,,P,10-4,,gene_product,taxon:7227,20060803,FlyBase,,
4,FB,FBgn0026615,10-4,,GO:0003674,FB:FBrf0159398|GO_REF:0000015,ND,,F,10-4,,gene_product,taxon:7227,20060803,FlyBase,,


In [11]:
df_process = df_clean[df_clean["GO class"] == 'P'].copy()

In [12]:
df_process.shape

(55579, 17)

In [13]:
# remove any annotation assigned by electronic matching and with the NOT qualifier which is used to specify a gene is not associated with a term
df_process = df_process[df_process["Evidence code"] != 'IEA']
df_process = df_process[df_process["Qualifier"] != 'NOT']

In [14]:
df_process = df_process[['Gene symbol', 'GO ID']]
df_process.reset_index(inplace=True)
df_process = df_process.drop(columns = 'index')

In [15]:
df_process.shape

(52028, 2)

## load GO digraph

In [16]:
digraph = goenrich.obo.ontology('input/go-basic.ob')

## only keep GO IDs with depth =>4, replace others with NaN and then remove those rows

In [17]:
## code copied from Gene Ontology (GO) Biological Process by Moshe Silverstein 

lst = []

for i,index in enumerate(df_process.index):
    
    progressPercent = ((i+1)/len(df_process.index))*100

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df_process.index)))
    sys.stdout.flush()
    
    term = df_process.loc[index, 'GO ID']
    if term in digraph.node:
        if digraph.node[term]['depth'] >= 4:
            lst.append(term)
        else:
            lst.append(np.nan)
    else:
        lst.append(np.nan)
           
df_process['GO ID'] = lst

In [18]:
df_process.shape

(52028, 2)

In [19]:
df_process.dropna(inplace=True)

In [20]:
df_process.shape

(39794, 2)

In [21]:
df_process.head()

,Gene symbol,GO ID
2,14-3-3epsilon,GO:0000077
3,14-3-3epsilon,GO:0000077
4,14-3-3epsilon,GO:0007444
7,14-3-3epsilon,GO:0007088
8,14-3-3epsilon,GO:0000077


## term propagation-propergate child gene term relationships to parent terms

In [22]:
## code copied from Gene Ontology (GO) Biological Process by Moshe Silverstein 
lst1 = []
lst2 = []

for i,index in enumerate(df_process.index):
    
#     progressPercent = ((i+1)/len(df.index))*100

#     sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df.index)))
#     sys.stdout.flush()
    
#     term = df.loc[index, 'GO ID']
#     for parent in digraph.predecessors(term):
#         if parent in digraph.node:
#             if digraph.node[parent]['depth'] >= 4:
#                 lst1.append(df.loc[index, 'DB Object Symbol'])
#                 lst2.append(parent)
#                 print(term, parent)
    term = df_process.loc[index, 'GO ID']
    for parent in digraph.successors(term):
        if parent in digraph.node:
            if digraph.node[parent]['depth'] >= 4:
                lst1.append(df_process.loc[index, 'Gene symbol'])
                lst2.append(parent)
#                 print(term, parent)


temp = pd.DataFrame()
temp['Gene symbol'] = lst1
temp['GO ID']  = lst2
df_process = pd.concat([df_process, temp])
df_process.reset_index(inplace=True)
df_process = df_process.drop(columns = 'index')

In [23]:
df_process.shape

(100560, 2)

In [24]:
df_process.head()

,Gene symbol,GO ID
0,14-3-3epsilon,GO:0000077
1,14-3-3epsilon,GO:0000077
2,14-3-3epsilon,GO:0007444
3,14-3-3epsilon,GO:0007088
4,14-3-3epsilon,GO:0000077


## map GO ID to descriptive name

In [25]:
## code copied from Gene Ontology (GO) Biological Process by Moshe Silverstein 
lst = []

for i,index in enumerate(df_process.index):
    
    progressPercent = ((i+1)/len(df_process.index))*100

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df_process.index)))
    sys.stdout.flush()
    
    lst.append(str(digraph.node[df_process.loc[index, 'GO ID']]['name'])+' '+ '('+str(df_process.loc[index, 'GO ID'])+')')
    
df_process['GO ID'] = lst

In [26]:
df_process.head()

,Gene symbol,GO ID
0,14-3-3epsilon,DNA damage checkpoint (GO:0000077)
1,14-3-3epsilon,DNA damage checkpoint (GO:0000077)
2,14-3-3epsilon,imaginal disc development (GO:0007444)
3,14-3-3epsilon,regulation of mitotic nuclear division (GO:000...
4,14-3-3epsilon,DNA damage checkpoint (GO:0000077)


In [27]:
df_process.drop_duplicates(inplace=True)

In [28]:
df_process.head()

,Gene symbol,GO ID
0,14-3-3epsilon,DNA damage checkpoint (GO:0000077)
2,14-3-3epsilon,imaginal disc development (GO:0007444)
3,14-3-3epsilon,regulation of mitotic nuclear division (GO:000...
5,14-3-3epsilon,wing disc dorsal/ventral pattern formation (GO...
6,14-3-3epsilon,positive regulation of growth (GO:0045927)


In [29]:
df_process.reset_index(inplace=True)

In [30]:
df_process.head()

,index,Gene symbol,GO ID
0,0,14-3-3epsilon,DNA damage checkpoint (GO:0000077)
1,2,14-3-3epsilon,imaginal disc development (GO:0007444)
2,3,14-3-3epsilon,regulation of mitotic nuclear division (GO:000...
3,5,14-3-3epsilon,wing disc dorsal/ventral pattern formation (GO...
4,6,14-3-3epsilon,positive regulation of growth (GO:0045927)


In [31]:
df_process = df_process.drop(columns = 'index')

In [32]:
df_process.head()

,Gene symbol,GO ID
0,14-3-3epsilon,DNA damage checkpoint (GO:0000077)
1,14-3-3epsilon,imaginal disc development (GO:0007444)
2,14-3-3epsilon,regulation of mitotic nuclear division (GO:000...
3,14-3-3epsilon,wing disc dorsal/ventral pattern formation (GO...
4,14-3-3epsilon,positive regulation of growth (GO:0045927)


In [33]:
df_process.shape

(75052, 2)

In [34]:
number_terms = len(df_process.iloc[:, 1].unique())
number_terms

6060

## build the protein-coding gene list

In [38]:
dMelanogaster_geneInfo = pd.read_csv("input/Drosophila_melanogaster.gene_info", sep="\t")

In [39]:
dMelanogaster_geneInfo.head()

,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type
0,7227,30970,CG3038,Dmel_CG3038,Dmel\CG3038|EG:BACR37P7.1|GalT3|dbeta3GnT,FLYBASE:FBgn0040373,X,1A1-1A1|1-0 cM,uncharacterized protein,protein-coding,CG3038,-,O,uncharacterized protein|CG3038-PA|CG3038-PB|CG...,20180601,-
1,7227,30971,G9a,Dmel_CG2995,CG2995|Dmel\CG2995|EG:BACR37P7.2|EHMT|cab65850...,FLYBASE:FBgn0040372,X,1A1-1A1|1-0 cM,G9a,protein-coding,G9a,-,O,G9a|CG2995-PA|CG2995-PB|G9a-PA|G9a-PB|dG9a|euc...,20180603,-
2,7227,30972,CG13377,Dmel_CG13377,BG01596|Dmel\CG13377|EG:BACR37P7.9,FLYBASE:FBgn0261446,X,1A1-1A1|1-0 cM,uncharacterized protein,protein-coding,CG13377,-,O,uncharacterized protein|BG01596|CG13377-PA|CG1...,20180603,-
3,7227,30973,cin,Dmel_CG2945,CG2945|CIN|Dmel\CG2945|EG:BACR37P7.3|fs(1)M50|...,FLYBASE:FBgn0000316,X,1A1-1A1|1-0 cM,cinnamon,protein-coding,cin,cinnamon,O,cinnamon|CG2945-PA|CG2945-PB|cin-PA|cin-PB,20180603,-
4,7227,30975,ewg,Dmel_CG3114,CG3114|Dmel\CG3114|EC3|EG:BACR37P7.7|EWG|Ewg|N...,FLYBASE:FBgn0005427,X,1A1-1A1|1-0 cM,erect wing,protein-coding,ewg,erect wing,O,erect wing|CG3114-PA|CG3114-PB|CG3114-PC|CG311...,20180603,-


In [40]:
dMelanogaster_geneInfo.shape

(25030, 16)

In [41]:
dMelanogaster_proteinCoding = dMelanogaster_geneInfo[dMelanogaster_geneInfo["type_of_gene"] == "protein-coding"]

In [42]:
dMelanogaster_proteinCoding.shape

(13931, 16)

## only keep gene symbolys that are in the S cerevisiae table

In [43]:
df_process_protein = df_process.loc[df_process["Gene symbol"].isin(dMelanogaster_proteinCoding["Symbol"].unique())]

In [44]:
df_process_protein.shape

(69273, 2)

In [45]:
filename = 'process_gene_ontology_flybase_greater4_%s.gmt'% str(datetime.date.today())[0:7].replace('-', '_')
file = open(filename,'w+') 
terms = df_process_protein["GO ID"].unique()
for i,term in enumerate(terms):
    
    progressPercent = ((i+1)/len(terms))*100
    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(terms)))
    sys.stdout.flush()
            
    df_byTerm = df_process_protein.loc[df_process_protein["GO ID"] == term]
    
    if df_byTerm.shape[0] > 4:
        # split splice variant names
        split_splice = lambda x: x.split('.')[0]
        df_byTerm["Gene symbol"] = df_byTerm["Gene symbol"].apply(split_splice)
        
        
        if len(df_byTerm["Gene symbol"].unique()) > 4:
            file.write("%s\t" % term)
            file.write("\t")
            genes = df_byTerm["Gene symbol"].unique()
    
            for gene in genes:
                file.write("%s\t" % gene)
            file.write("\n")
        
file.close()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


## get stats for gmt

In [49]:
def getTermStats(loaded_gmt, geneInfoTable, num_samplings, num_genes_in_sampling) :
    # get the relevant terms and set a dictionary for them
    term_set = loaded_gmt.iloc[:,0].apply(lambda x: x.split("\t")[0])
    term_genes_dict = {term_set[i]: loaded_gmt.iloc[i,0].split("\t")[2:] for i in range(0, loaded_gmt.shape[0])}
    term_rank_dict = {k: [] for k in term_set}
    term_stats_dict = {k: [] for k in term_set}
    
    # get the total number of genes in the organism
    n_genes = len(geneInfoTable["Symbol"].unique())
    
    # number of genes per sampling, number of times to sample
    genes_in_sampleing = num_genes_in_sampling
    num_samplings = num_samplings
    
    for i,x in enumerate(range(0,num_samplings)):
        
        progressPercent = ((i+1)/len(range(0,num_samplings)))*100
        
        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(range(0,num_samplings))))
        sys.stdout.flush()
            
        
        randomSet = set(geneInfoTable["Symbol"].sample(genes_in_sampleing).apply(lambda x: str(x).split('.')[0]))
        
        # initialize an array to hold the pvalue for each term
        pvals = np.array([])
        for key in term_rank_dict.keys():
            
            termSet = set(term_genes_dict[key])
            # calculate p value using fisher exact test
            # implemented using the formula found in the GeneOverap bioconductor package for R
            pval_term = stat.fisher_exact([[n_genes - len(termSet.union(randomSet)), len(randomSet.difference(termSet))], [len(termSet.difference(randomSet)), len(termSet.intersection(randomSet))]])[1]
            pvals = np.append(pvals, [pval_term])
        
        # sort the pvals and add the rank to the term_rank dict
        sorted_pvals = pvals.argsort()
        
        # find the rank for each term by sorting the pvals array and getting the index of the key position in the
        # sorted list. Need to +1 because the index begins with 0
        
        for j,key in enumerate(term_rank_dict.keys()):
            term_rank_dict[key].append(np.where(sorted_pvals == j)[0][0] + 1)
            
    for key in term_rank_dict:
        term_stats_dict[key] = [np.mean(term_rank_dict[key]), np.std(term_rank_dict[key])]
            
    
    
    return term_stats_dict

In [47]:
process_GO_gmt = pd.read_csv("output/process_gene_ontology_flybase_greater4_2018_06.gmt", sep="~", header=None)
type(process_GO_gmt)

pandas.core.frame.DataFrame

In [50]:
iterations_200 = getTermStats(process_GO_gmt, dMelanogaster_proteinCoding, num_samplings=200, num_genes_in_sampling=300)

In [51]:
df_200_iterations = pd.DataFrame.from_dict(iterations_200, orient='index', columns = ['mean', 'sd'])

In [52]:
df_200_iterations.to_csv('output/process_gene_ontology_flybase_greater4_2018_06_stats.tsv', sep='\t', header=False)